# CasADi vs Theano

###### Jupyter notebook to compare the speed of Theano and CasADi softwares of automatic differentiation on the specific task of building force kernels

###### The TwoBody is fully implemented in both softwares and Theano appears to be sensibly faster. Probably this is due to the use of numpy arrays

# Two Body

## Theano 

In [3]:
import theano.tensor as T
from theano import function, scan
import numpy as np

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [4]:
# Theano functions

def compile_twobody_singlespecies():
	"""
	This function generates theano compiled kernels for energy and force learning

	The position of the atoms relative to the centrla one, and their chemical species
	are defined by a matrix of dimension Mx5

	Returns:
		k2_ee (func): energy-energy kernel
		k2_ef (func): energy-force kernel
		k2_ff (func): force-force kernel
	"""
	print("Started compilation of theano two body single species kernels")
	# --------------------------------------------------
	# INITIAL DEFINITIONS
	# --------------------------------------------------

	# positions of central atoms
	r1, r2 = T.dvectors('r1d', 'r2d')
	# positions of neighbours
	rho1, rho2 = T.dmatrices('rho1', 'rho2')
	# lengthscale hyperparameter
	sig = T.dscalar('sig')
	# cutoff hyperparameters
	theta = T.dscalar('theta')
	rc = T.dscalar('rc')

	# positions of neighbours without chemical species (3D space assumed)
	rho1s = rho1#[:, 0:3]
	rho2s = rho2#[:, 0:3]

	# --------------------------------------------------
	# RELATIVE DISTANCES TO CENTRAL VECTOR
	# --------------------------------------------------

	# first and second configuration
	r1j = T.sqrt(T.sum((rho1s[:, :] - r1[None, :])** 2, axis=1))
	r2m = T.sqrt(T.sum((rho2s[:, :] - r2[None, :])** 2, axis=1))

	# Cutoff function
	k_ij = T.exp(-(r1j[:, None] - r2m[None, :])** 2 /(2*sig**2))

	cut_ij = (0.5 * (1 + T.sgn(rc - r1j[:, None]))) * (0.5 * (1 + T.sgn(rc - r2m[None, :]))) * \
	         (T.exp(-theta / (rc - r1j[:, None])) * T.exp(-theta / (rc - r2m[None, :])))

	k_ij = k_ij #* cut_ij

	# kernel
	k = T.sum(k_ij)

	# --------------------------------------------------
	# FINAL FUNCTIONS
	# --------------------------------------------------

	# energy energy kernel
	k_ee_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k, allow_input_downcast=False, on_unused_input='ignore')

	# energy force kernel
	k_ef = T.grad(k, r2)
	k_ef_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ef, allow_input_downcast=False, on_unused_input='ignore')

	# force force kernel
	k_ff = T.grad(k, r1)
	k_ff_der, updates = scan(lambda j, k_ff, r2: T.grad(k_ff[j], r2),
	                         sequences=T.arange(k_ff.shape[0]), non_sequences=[k_ff, r2])

	k_ff_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ff_der, allow_input_downcast=False,
	                    on_unused_input='ignore')

	# --------------------------------------------------
	# WRAPPERS (we don't want to plug the position of the central element every time)
	# --------------------------------------------------

	def k2_ee(conf1, conf2, sig, theta, rc):
		"""
		Two body kernel for energy-energy correlation

		Args:
			conf1: first configuration.
			conf2: second configuration.
			sig: lengthscale hyperparameter.
			theta: cutoff smoothness hyperparameter.
			rc: cutoff distance hyperparameter.

		Returns:
			kernel (scalar):

		"""
		return k_ee_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

	def k2_ef(conf1, conf2, sig, theta, rc):
		"""
		Two body kernel for energy-force correlation

		Args:
			conf1: first configuration.
			conf2: second configuration.
			sig: lengthscale hyperparameter.
			theta: cutoff smoothness hyperparameter.
			rc: cutoff distance hyperparameter.

		Returns:
			kernel (vector):
		"""

		return k_ef_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

	def k2_ff(conf1, conf2, sig, theta, rc):
		"""
		Two body kernel for force-force correlation

		Args:
			conf1: first configuration.
			conf2: second configuration.
			sig: lengthscale hyperparameter.
			theta: cutoff smoothness hyperparameter.
			rc: cutoff distance hyperparameter.

		Returns:
			kernel (matrix):
		"""

		return k_ff_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

	print("Ended compilation of theano two body single species kernels")

	return k2_ee, k2_ef, k2_ff



In [5]:
k2_ee, k2_ef, k2_ff = compile_twobody_singlespecies()

Started compilation of theano two body single species kernels
Ended compilation of theano two body single species kernels


## CasADi

In [6]:
from casadi import *

In [7]:
# fix the number of neighbours
M = 100

In [8]:
# --------------------------------------------------
# INITIAL DEFINITIONS
# --------------------------------------------------

r1, r2 = SX.sym( 'r1', 1, 3  ), SX.sym( 'r2', 1, 3 )
rho1, rho2 = SX.sym('rho1', M, 3 ), SX.sym('rho2', M, 3 )
sig = SX.sym('sig')

In [9]:
# kernel
k = SX.zeros()
for i in range(M):
    for j in range(M):
        k = k + exp(-(norm_2(rho1[i, :] - r1) - norm_2(rho2[j, :] - r2))**2/(2*sig**2))

In [10]:
# single and double derivative
grad = gradient(k, r1)
hess = jacobian( grad , r2)

In [11]:
# define CasADi functions
k2_ee_c = Function('k2_ee_c', [rho1, rho2, r1, r2, sig], [k], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k' ] )
k2_ef_c = Function('k2_ef_c', [rho1, rho2, r1, r2, sig], [grad], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k_g' ] )
k2_ff_c = Function('k2_ff_c', [rho1, rho2, r1, r2, sig], [hess], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k_h' ] )

In [12]:
# generate a CasADi C-compiled function
k2_ff_c.generate('k2_ff_c.c')


'k2_ff_c.c'

In [13]:
# compile CasADi function
!gcc -shared -fPIC k2_ff_c.c -o k2_ff_c.so

In [14]:
# import C-compiled function
k2_ff_c_comp = external( 'k2_ff_c', 'k2_ff_c.so' )

In [15]:
# define a benchmarking test on random configurations
%timeit
n = 10
d = 3
nn = M
confs = (np.random.rand(n, nn, d)-0.5)*100


def fill_gram_theano():
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k2_ff(confs[i],confs[j], 1., 1., 1.)
    return gram

def fill_gram_casadi():
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k2_ff_c(confs[i], confs[j], np.zeros(3), np.zeros(3), 1.)
    return gram

def fill_gram_casadi_comp():
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k2_ff_c_comp(confs[i],confs[j], np.zeros(3), np.zeros(3), 1.)
    return gram

In [16]:
# check that results are identical

print(k2_ff(confs[0],confs[1], 1., 100000., 1.))
print(k2_ff_c(rho1 = confs[0], rho2 = confs[1], r1 = np.zeros(3), r2 = np.zeros(3), sig = 1.))
print(k2_ff_c_comp(rho1 = confs[0], rho2 = confs[1], r1 = np.zeros(3), r2 = np.zeros(3), sig = 1.))

[[-2.52526964  0.79162227 -2.77993914]
 [-1.92510829 -6.42315901 -0.12114935]
 [-4.88533568 13.64772089 -3.64144628]]
{'k_h': DM(
[[-2.52527, 0.791622, -2.77994], 
 [-1.92511, -6.42316, -0.121149], 
 [-4.88534, 13.6477, -3.64145]])}
{'k_h': DM(
[[-2.52527, 0.791622, -2.77994], 
 [-1.92511, -6.42316, -0.121149], 
 [-4.88534, 13.6477, -3.64145]])}


In [17]:
gram1, gram2, gram3 = fill_gram_theano(), fill_gram_casadi(), fill_gram_casadi_comp()
np.allclose(gram1, gram2) and np.allclose(gram2, gram3)

True

In [18]:
# profiling of the three versions
%timeit fill_gram_theano()
%timeit fill_gram_casadi()
%timeit fill_gram_casadi_comp()

60.8 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
328 ms ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
239 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Theano is sensibly faster than the others

## Three Body

In [19]:
# TO BE FINISHED

In [20]:
def compile_threebody_singlespecies():
	"""
	This function generates theano compiled kernels for energy and force learning

	The position of the atoms relative to the centrla one, and their chemical species
	are defined by a matrix of dimension Mx5

	Returns:
		k3_ee (func): energy-energy kernel
		k3_ef (func): energy-force kernel
		k3_ff (func): force-force kernel
	"""

	print("Started compilation of theano three body single species kernels")

	# --------------------------------------------------
	# INITIAL DEFINITIONS
	# --------------------------------------------------

	# positions of central atoms
	r1, r2 = T.dvectors('r1d', 'r2d')
	# positions of neighbours
	rho1, rho2 = T.dmatrices('rho1', 'rho2')
	# hyperparameter
	sig = T.dscalar('sig')
	# cutoff hyperparameters
	theta = T.dscalar('theta')
	rc = T.dscalar('rc')

	# positions of neighbours without chemical species

	#rho1s = rho1[:, 0:3]
	#rho2s = rho2[:, 0:3]

	# --------------------------------------------------
	# RELATIVE DISTANCES TO CENTRAL VECTOR AND BETWEEN NEIGHBOURS
	# --------------------------------------------------

	# first and second configuration
	r1j = T.sqrt(T.sum((rho1s[:, :] - r1[None, :]) ** 2, axis=1))
	r2m = T.sqrt(T.sum((rho2s[:, :] - r2[None, :]) ** 2, axis=1))
	rjk = T.sqrt(T.sum((rho1s[None, :, :] - rho1s[:, None, :]) ** 2, axis=2))
	rmn = T.sqrt(T.sum((rho2s[None, :, :] - rho2s[:, None, :]) ** 2, axis=2))

	# --------------------------------------------------
	# BUILD THE KERNEL
	# --------------------------------------------------

	# Squared exp of differences
	se_1j2m = T.exp(-(r1j[:, None] - r2m[None, :]) ** 2 / (2 * sig ** 2))
	se_jkmn = T.exp(-(rjk[:, :, None, None] - rmn[None, None, :, :]) ** 2 / (2 * sig ** 2))
	se_jk2m = T.exp(-(rjk[:, :, None] - r2m[None, None, :]) ** 2 / (2 * sig ** 2))
	se_1jmn = T.exp(-(r1j[:, None, None] - rmn[None, :, :]) ** 2 / (2 * sig ** 2))

	# Kernel not summed (cyclic permutations)
	k1n = (se_1j2m[:, None, :, None] * se_1j2m[None, :, None, :] * se_jkmn)
	k2n = (se_1jmn[:, None, :, :] * se_jk2m[:, :, None, :] * se_1j2m[None, :, :, None])
	k3n = (se_1j2m[:, None, None, :] * se_jk2m[:, :, :, None] * se_1jmn[None, :, :, :])

	# final shape is M1 M1 M2 M2

	ker_jkmn = (k1n + k2n + k3n)

	cut_ik = (T.exp(-theta / T.abs_(rc - r1j[:, None])) *
	          T.exp(-theta / T.abs_(rc - r1j[None, :])) *
	          T.exp(-theta / T.abs_(rc - rjk[:, :])) *
	          (0.5 * (T.sgn(rc - r1j) + 1))[None, :] *
	          (0.5 * (T.sgn(rc - r1j) + 1))[:, None] *
	          (0.5 * (T.sgn(rc - rjk) + 1))[:, :])

	cut_mn = (T.exp(-theta / T.abs_(rc - r2m[:, None])) *
	          T.exp(-theta / T.abs_(rc - r2m[None, :])) *
	          T.exp(-theta / T.abs_(rc - rmn[:, :])) *
	          (0.5 * (T.sgn(rc - r2m) + 1))[None, :] *
	          (0.5 * (T.sgn(rc - r2m) + 1))[:, None] *
	          (0.5 * (T.sgn(rc - rmn) + 1))[:, :])

	ker_jkmn_withcutoff = ker_jkmn * cut_ik[:, :, None, None] * cut_mn[None, None, :, :]

	# --------------------------------------------------
	# REMOVE DIAGONAL ELEMENTS
	# --------------------------------------------------

	mask_jk = T.ones_like(rjk) - T.identity_like(rjk)
	mask_mn = T.ones_like(rmn) - T.identity_like(rmn)

	mask_jkmn = mask_jk[:, :, None, None] * mask_mn[None, None, :, :]

	k_cutoff = T.sum(ker_jkmn_withcutoff * mask_jkmn)

	# --------------------------------------------------
	# FINAL FUNCTIONS
	# --------------------------------------------------

	# energy energy kernel
	k_ee_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_cutoff, on_unused_input='warn')

	# energy force kernel
	k_ef_cut = T.grad(k_cutoff, r2)
	k_ef_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ef_cut, on_unused_input='warn')

	# force force kernel
	k_ff_cut = T.grad(k_cutoff, r1)
	k_ff_cut_der, updates = scan(lambda j, k_ff_cut, r2: T.grad(k_ff_cut[j], r2),
	                             sequences=T.arange(k_ff_cut.shape[0]), non_sequences=[k_ff_cut, r2])
	k_ff_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ff_cut_der, on_unused_input='warn')

	# WRAPPERS (we don't want to plug the position of the central element every time)

	def k3_ee(conf1, conf2, sig, theta, rc):
		"""
		Two body kernel for energy-energy correlation

		Args:
			conf1: first configuration.
			conf2: second configuration.
			sig: lengthscale hyperparameter.
			theta: cutoff smoothness hyperparameter.
			rc: cutoff distance hyperparameter.

		Returns:
			kernel (scalar):

		"""
		return k_ee_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

	def k3_ef(conf1, conf2, sig, theta, rc):
		"""
		Two body kernel for energy-force correlation

		Args:
			conf1: first configuration.
			conf2: second configuration.
			sig: lengthscale hyperparameter.
			theta: cutoff smoothness hyperparameter.
			rc: cutoff distance hyperparameter.

		Returns:
			kernel (vector):
		"""

		return k_ef_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

	def k3_ff(conf1, conf2, sig, theta, rc):
		"""
		Two body kernel for force-force correlation

		Args:
			conf1: first configuration.
			conf2: second configuration.
			sig: lengthscale hyperparameter.
			theta: cutoff smoothness hyperparameter.
			rc: cutoff distance hyperparameter.

		Returns:
			kernel (matrix):
		"""

		return k_ff_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

	print("Ended compilation of theano three body single species kernels")

	return k3_ee, k3_ef, k3_ff

In [21]:
k3_ee, k3_ef, k3_ff = compile_twobody_singlespecies()

Started compilation of theano two body single species kernels
Ended compilation of theano two body single species kernels


In [22]:
# --------------------------------------------------
# INITIAL DEFINITIONS
# --------------------------------------------------

# positions of central atoms
M = 3
r1, r2 = SX.sym( 'r1', 1, 3  ), SX.sym( 'r2', 1, 3 )
d1, d2 = SX.sym( 'd1'  ), SX.sym( 'd2')
rho1, rho2 = SX.sym('rho1', M, 3 ), SX.sym('rho2', M, 3 )
sig = SX.sym('sig')
#theta = SX.sym('theta')
#rc = SX.sym('rc')
#rho1, rho2 = MX.sym('rho1', 3 ), MX.sym( 'rho2')


                

In [ ]:
k3 = SX.zeros()
for i1 in range(M):
    for j1 in range(M):
        for i2 in range(M):
            for j2 in range(M):
                
                ri, rj, rij = norm_2(rho1[i, :] - r1), norm_2(rho1[j, :] - r1), norm_2(rho1[i, :] - rho1[j, :])
                ri2, rj2, ri2j2 = norm_2(rho2[i2, :] - r2), norm_2(rho2[j2, :] - r2), norm_2(rho2[i2, :] - rho2[j2, :])
                
                k3 = k3 + exp(-(ri - ri2)**2/(2*sig**2)) * \
                    exp(-(rj - rj2)**2/(2*sig**2)) * \
                    exp(-(rij - ri2j2)**2/(2*sig**2))
                

In [ ]:
grad3 = gradient(k3, r1)
hess3 = jacobian( grad , r2)

In [ ]:
k3_ee_c = Function('k2_ee_c', [rho1, rho2, r1, r2, sig], [k3], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k3' ] )
k3_ef_c = Function('k2_ef_c', [rho1, rho2, r1, r2, sig], [grad3], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k3_g' ] )
k3_ff_c = Function('k2_ff_c', [rho1, rho2, r1, r2, sig], [hess3], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k3_h' ] )

In [ ]:
def fill_gram_theano3():
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k3_ff(confs[i],confs[j], 1., 1., 1.)
    return gram

def fill_gram_casadi3():
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k3_ff_c(confs[i], confs[j], np.zeros(3), np.zeros(3), 1.)
    return gram

In [ ]:
%timeit fill_gram_theano3()

In [ ]:
%timeit fill_gram_casadi3()